In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# !ls /content/drive/MyDrive/Computadora\ Virtual/Data\ Analytics/Investigacion/Alzheimer/Data/Preprocessed/Experimento\ 03/
# !ls /content/drive/My\ Drive/Tesis/Data/New_FCBF/graficos/PCA_OTH/Depression_PCA_labels/
!ls /content/drive/My\ Drive/JnOTE/graficos/PCA_OTH/Depression_PCA_labels/

oth0.06_ith0.5_PCA_Em0_nf100_bwf1_fi0.5_ov0_bdfb20_Depression_Expression.csv
oth0.06_ith0.5_PCA_Em0_nf100_bwf1_fi0.5_ov0_bdfb25_Depression_Expression.csv
oth0.06_ith0.5_PCA_Em0_nf100_bwf1_fi0.5_ov0_bdfb30_Depression_Expression.csv
oth0.06_ith0.5_PCA_Em0_nf100_bwf1_fi0.5_ov0_bdfb35_Depression_Expression.csv
oth0.06_ith0.5_PCA_Em0_nf100_bwf1_fi0.5_ov0_bdfb40_Depression_Expression.csv
oth0.06_ith0.5_PCA_Em0_nf100_bwf1_fi0.5_ov0_bdfb45_Depression_Expression.csv
oth0.06_ith0.5_PCA_Em0_nf100_bwf1_fi0.5_ov0_bdfb50_Depression_Expression.csv
oth0.06_ith0.5_PCA_Em0_nf100_bwf1_fi0.5_ov0_bdfb55_Depression_Expression.csv
oth0.06_ith0.5_PCA_Em0_nf100_bwf1_fi0.5_ov0_bdfb60_Depression_Expression.csv
oth0.08_ith0.5_PCA_Em0_nf100_bwf1_fi0.5_ov0_bdfb20_Depression_Expression.csv
oth0.08_ith0.5_PCA_Em0_nf100_bwf1_fi0.5_ov0_bdfb25_Depression_Expression.csv
oth0.08_ith0.5_PCA_Em0_nf100_bwf1_fi0.5_ov0_bdfb30_Depression_Expression.csv
oth0.08_ith0.5_PCA_Em0_nf100_bwf1_fi0.5_ov0_bdfb35_Depression_Expression.csv

# Import libraries

In [ ]:
!pip install h2o

     |████████████████████████████████| 164.8MB 87kB/s 
  Created wheel for h2o: filename=h2o-3.32.1.3-py2.py3-none-any.whl size=164854343 sha256=33872b05329773ff1ecfc760b27700295a4d0ab86c014534c6d417a3f7e87d26
  Stored in directory: /root/.cache/pip/wheels/4f/bb/82/0988682f9abd0b5e95109a4aae6793dd427173b6dc671d9252
Successfully built h2o


REINICIAR ENTORNO DE EJECUCIÓN

In [ ]:
for name in dir():
       if not name.startswith('_'):
           del globals()[name]

# CONTINUAR

In [ ]:
import pandas as pd
import numpy as np

from random import seed
from random import randint
import random

import time

In [ ]:
import h2o
h2o.init()
from h2o.automl import H2OAutoML

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpfdxhit5r
  JVM stdout: /tmp/tmpfdxhit5r/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpfdxhit5r/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,1 month and 2 days
H2O_cluster_name:,H2O_from_python_unknownUser_2chmmn
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.174 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


#  REINICIAR ENTORNO DE EJECUCIÓN

# Training and Testing Function

In [ ]:
def agument_Data(data, Eps, proportion):
  print('proportion=', proportion)
  N = int(proportion*data.shape[0])
  df_to_be_agumented = data
  rn = rng.normal(0, Eps, size=(1, data.shape[1]))[0]
  #print('rn=',rn,'len(rn)',len(rn))

  for i in range(N):                              # create new N data. Where N are the required proportion.
    rd_row=random.randint(0,len(data)-1)             # randomly select one row to include noise.
    #print('i',i,'rd_row',rd_row)
    #print('data.values[rd_row,0:data.shape[1]]=',data.values[rd_row,0:data.shape[1]])
    b = data.values[rd_row,0:data.shape[1]] + rn  # add Gaussian Noise to the selected observation.
    #print('1b=',b)
    b = pd.DataFrame(b)                      # return format to Panda Dataframe
    b=b.T
    #print('2b=',b)

    b.columns = data.columns                         # re-name the new dataframe with the original names of the columns
    #print('3b=',b)
    b.values[0,b.shape[1]-1] = int(data.values[rd_row,b.shape[1]-1]) # convert to integer the last value (this is the value used to classify)
    #print('4b=',b)
    df_to_be_agumented = pd.concat([df_to_be_agumented,b])      # concat the augmente observation at the end of the array.
    df_to_be_agumented.reset_index(inplace=True, drop=True)     # reset the index
  return(df_to_be_agumented)

In [ ]:
def make_dataframes_classes_same_length(training_data, Eps, prop):  # data must be the training data
  e1 = training_data[(training_data['Depression_Expression']==0)]  # data presenting the emotion.
  e0 = training_data[(training_data['Depression_Expression']==1)]   # data no presenting the emotion.
  l_e0 = len(e0)
  l_e1 = len(e1)
  print('e_e0=',len(e0),'e_e1=',len(e1))
  if l_e0>l_e1:
      prop = l_e0/l_e1-1
      e1=agument_Data(e1, Eps, prop)
  else:
      prop = l_e1/l_e0-1
      e0=agument_Data(e0, Eps, prop)
  df_to_be_agumented = pd.concat([e0,e1])                   # concat the augmente observation at the end of the array.
  df_to_be_agumented.reset_index(inplace=True, drop=True)   # reset the index
  return(df_to_be_agumented)

In [ ]:
def train_test_LOOnd(data, test_ctl_pd, n, num_testing_samples):

  print('test_ctl_pd.shape[0]=', test_ctl_pd.shape[0])
  #part_length = int(test_ctl_pd.shape[0]/num_testing_samples) 
  #print('part_length', part_length)

  test  = test_ctl_pd.iloc[:num_testing_samples]                # get the top "part_length" users for testing.
  # Constructing the training data:
  train1 = data.iloc[                         :num_testing_samples*n]  # get the top "part_length" users for training.
  train2 = data.iloc[num_testing_samples*(n+1):                     ]  # get the bottom "part_length" users for traing.
  train = pd.concat([train1, train2],axis=0)                           # concatenatig the two parts to form the training data.
  
  test_ctl_pd = test_ctl_pd.iloc[num_testing_samples:]        # delete the top "part_length" from the train_ctl_pd for the next iteration.
  
  print('test.shape[0] =', test.shape[0], 'train.shape[0]=', train.shape[0])

  return(train, test, test_ctl_pd)    

# **Main**

In [ ]:
####################################################################################
####################################################################################
####################################################################################
start_time=time.time()
path = "/content/drive/My Drive/JnOTE/graficos/"
name='oth0.06_ith0.5_PCA_Em0_nf100_bwf1_fi0.5_ov0_bdfb25_Depression_Expression'
file= path + "PCA_OTH/Depression_PCA_labels/" + name + ".csv"

#!ls /content/drive/MyDrive/Computadora\ Virtual/Data\ Analytics/Investigacion/Alzheimer/Data/Preprocessed/Experimento\ 03/


results_file = open(path + 'Results/' + name + '.txt', 'a')

#PaHaw_data = pd.read_csv(file)
data = pd.read_csv(file)
data = data.dropna(axis=1)      # Dropping the columns having NaN/NaT values
#data=data.iloc[np.random.permutation(data.index)].reset_index(drop=True) # Randomly Shuffle DataFrame Rows to avoid one of the folds only contains one category.

#data=data._get_numeric_data()  # keep only columns which all values are numeric.
h2o_df = h2o.H2OFrame(data)     # convert panda dataframe to use the h2o machine learning plataform.
h2o.no_progress()
#h2o.show_progress()

nf = data.shape[1]-1  # número de features (componentes)
nc = nf               # número de componetes (features) a utilizar
col = list(range(nc))

per_testing_set = 15     # Percentage of the testing rate.
prop_testing_set = per_testing_set/100
num_testing_samples = int(data.shape[0]*(prop_testing_set))   # Number of obervation used for testing.
print('num_testing_samples')

k=1
acc_max=0
for w in range(1,200):
   for j in range(w+1,w+10):
     print('&&&&&&&& w=',w,'j=',j, '&&&&&&&&&&&&&&&&&&&&&&&&&&&& Execution time: ', time.time()-start_time)
     for prop in [0.8]:
       print('prop=',prop)
       for Eps in [0.2]: # std deviation of the Gaussian Noise.
       #Eps=0.1
       #for SD in [5, 10, 15, 20, 25, 30]: # std deviation of the Gaussian Noise.
        print('Eps=',Eps)
        random.seed(w)
        rng=np.random.RandomState(j)
        acc_acu=0
        test_ctl_pd = data # initialize test_ctl_pd with all the users... in each iteration test_ctl_pd file will be deletting the testing files.
        i=0
        N = int(data.shape[0]/num_testing_samples)  # The number of iterations for co-evaluation depend on the num_obs (number of observations used for testing)
        print('N=',N,'data.shape[0]',data.shape[0],'num_testing_samples=',num_testing_samples)
        for n in range(0,N):
          print(i,'.',end='')
          i+=1
          start_time2=time.time()
          train, test, test_ctl_pd = train_test_LOOnd(data, test_ctl_pd, n, num_testing_samples)

          #print('train.shape[0]',train.shape[0])

          train = make_dataframes_classes_same_length(train, Eps, prop)
          train = agument_Data(train, Eps, prop)
          e1 = train[(train['Depression_Expression']==0)]   # data presenting DCL.
          e0 = train[(train['Depression_Expression']==1)]   # data presenting MA.
          l_e0 = len(e0)
          l_e1 = len(e1)
          #print('train.shape[0]',train.shape[0], 'l_e_MA=',l_e0,'l_e_DCLI=',l_e1)

          test.loc[(test.Depression_Expression)   == 0, 'Depression_Expression'] = 'yes'   # this assignment was needed, when the dataframe is convert to h2o dataframes.
          test.loc[(test.Depression_Expression)   == 1, 'Depression_Expression'] = 'no'
          train.loc[(train.Depression_Expression) == 0, 'Depression_Expression'] = 'yes'   # this assignment was needed, when the dataframe is convert to h2o dataframes.
          train.loc[(train.Depression_Expression) == 1, 'Depression_Expression'] = 'no'

          train_shuffled=train.iloc[np.random.permutation(train.index)].reset_index(drop=True) # Randomly Shuffle DataFrame Rows to avoid one of the folds only contains one category.

          test_h2o  = h2o.H2OFrame(test)            # convert panda dataframe to h20 dataframe
          train_h2o = h2o.H2OFrame(train_shuffled)           # convert panda dataframe to h20 dataframe

          print('time.time()-start_time=', time.time()-start_time2)
          #print('test_ctl_pd', test_ctl_pd, )

          # Identify predictors and response
          x = train_h2o.columns
          y = "Depression_Expression"
          x.remove(y)

          # model definition
          aml = H2OAutoML(max_runtime_secs=200,
                  max_models=15,
                  exclude_algos = ['GBM'],
                  seed=1,
                  nfolds=2,
                  stopping_metric="logloss", # logloss (this is the defalut for classification), MSE, MAE; RMSLE, AUC, AUCPR, lift_top_group, misclassification, maean_per_class_error
                  #sort_metric='auc',    # mse, MAE; RMSLE, auc, aucpr, lift_top_group, misclasification, maean_per_class_error
                  balance_classes=False # to minize the categories balance problem .
                  #project_name='Completed'
                  )
          
          # model trining
          #%time aml.train(x=x, y=y, training_frame=train)
          aml.train(x=x, y=y, training_frame=train_h2o)

          lb = aml.leaderboard
          #print('lb.head',lb.head(rows=lb.nrows))  # Print all rows instead of default (10 rows)

          #print('test_h2o',test_h2o)
          #acc=aml.model_performance(test_h2o).accuracy()
          #cma=aml.model_performance(test_h2o).confusion_matrix(metrics='accuracy')
          #print('cma=',cma)



          #print('i=',i,'test=',test)
          model = h2o.get_model(lb[0,0])      # Select the leader model to test.

          acc=model.model_performance(test_h2o).accuracy()[0][1]
          acc_acu += acc
          acc_m = acc_acu/i
          print('****** w=',w,'j=',j,'n=',n,'prop=',prop,'Eps=',Eps,'***********', 'acc',acc,'acc_m',acc_m, 'lb[0,0]=',lb[0,0], 'Execution time: ',time.time()-start_time)
        acc_m = acc_acu/(n+1)
        print(' ')
        print('******###################### w=',w,'j=',j,'prop=',prop,'Eps=',Eps,'***********','acc_m=',acc_m,'Execution time: ', time.time()-start_time)
        string_to_write = '****** w=' + str(w) + 'j=' + str(j) + 'prop=' + str(prop) + 'Eps=' + str(Eps) + "average accuracy=" + str(acc_m) + '**********' 'Execution time: ' + str(time.time()-start_time) + '\n'
        results_file.write(string_to_write)

        # View the AutoML Leaderboard
        #print('lb.head',lb.head(rows=lb.nrows))  # Print all rows instead of default (10 rows)
        # Get the "All Models" Stacked Ensemble model
        #se = aml.leader
        # Get the Stacked Ensemble metalearner model
        #metalearner = h2o.get_model(se.metalearner()['name']) # the goal of the meta learner is to ignore what is not important.
results_file.close()

Parse progress: |█████████████████████████████████████████████████████████| 100%
num_testing_samples
&&&&&&&& w= 1 j= 2 &&&&&&&&&&&&&&&&&&&&&&&&&&&& Execution time:  1.5935571193695068
prop= 0.8
Eps= 0.2
N= 6 data.shape[0] 128 num_testing_samples= 19
0 .test_ctl_pd.shape[0]= 128
test.shape[0] = 19 train.shape[0]= 109
e_e0= 26 e_e1= 83
proportion= 2.1923076923076925
proportion= 0.8


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


time.time()-start_time= 1.4157540798187256
****** w= 1 j= 2 n= 0 prop= 0.8 Eps= 0.2 *********** acc 0.8421052631578947 acc_m 0.8421052631578947 lb[0,0]= GLM_1_AutoML_20210621_193758 Execution time:  48.03995966911316
1 .test_ctl_pd.shape[0]= 109
test.shape[0] = 19 train.shape[0]= 109
e_e0= 29 e_e1= 80
proportion= 1.7586206896551726
proportion= 0.8
time.time()-start_time= 0.7666170597076416
****** w= 1 j= 2 n= 1 prop= 0.8 Eps= 0.2 *********** acc 1.0 acc_m 0.9210526315789473 lb[0,0]= StackedEnsemble_AllModels_AutoML_20210621_193844 Execution time:  83.94962882995605
2 .test_ctl_pd.shape[0]= 90
test.shape[0] = 19 train.shape[0]= 109
e_e0= 30 e_e1= 79
proportion= 1.6333333333333333
proportion= 0.8
time.time()-start_time= 0.846107006072998
****** w= 1 j= 2 n= 2 prop= 0.8 Eps= 0.2 *********** acc 1.0 acc_m 0.9473684210526315 lb[0,0]= StackedEnsemble_BestOfFamily_AutoML_20210621_193920 Execution time:  118.87752795219421
3 .test_ctl_pd.shape[0]= 71
test.shape[0] = 19 train.shape[0]= 109
e_e0

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


time.time()-start_time= 1.0876953601837158
****** w= 1 j= 3 n= 0 prop= 0.8 Eps= 0.2 *********** acc 1.0 acc_m 1.0 lb[0,0]= GLM_1_AutoML_20210621_194148 Execution time:  266.9501328468323
1 .test_ctl_pd.shape[0]= 109
test.shape[0] = 19 train.shape[0]= 109
e_e0= 29 e_e1= 80
proportion= 1.7586206896551726
proportion= 0.8
time.time()-start_time= 0.9147050380706787
****** w= 1 j= 3 n= 1 prop= 0.8 Eps= 0.2 *********** acc 0.9473684210526315 acc_m 0.9736842105263157 lb[0,0]= GLM_1_AutoML_20210621_194223 Execution time:  309.9946596622467
2 .test_ctl_pd.shape[0]= 90
test.shape[0] = 19 train.shape[0]= 109
e_e0= 30 e_e1= 79
proportion= 1.6333333333333333
proportion= 0.8
time.time()-start_time= 0.9092247486114502
****** w= 1 j= 3 n= 2 prop= 0.8 Eps= 0.2 *********** acc 0.9473684210526315 acc_m 0.9649122807017543 lb[0,0]= GLM_1_AutoML_20210621_194306 Execution time:  351.79628348350525
3 .test_ctl_pd.shape[0]= 71
test.shape[0] = 19 train.shape[0]= 109
e_e0= 30 e_e1= 79
proportion= 1.63333333333333

...

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


time.time()-start_time= 0.9338750839233398
****** w= 4 j= 11 n= 0 prop= 0.8 Eps= 0.2 *********** acc 0.7894736842105263 acc_m 0.7894736842105263 lb[0,0]= StackedEnsemble_AllModels_AutoML_20210621_223600 Execution time:  10749.73689699173
1 .test_ctl_pd.shape[0]= 109
test.shape[0] = 19 train.shape[0]= 109
e_e0= 29 e_e1= 80
proportion= 1.7586206896551726
proportion= 0.8
time.time()-start_time= 0.8172988891601562
****** w= 4 j= 11 n= 1 prop= 0.8 Eps= 0.2 *********** acc 1.0 acc_m 0.8947368421052632 lb[0,0]= GLM_1_AutoML_20210621_223705 Execution time:  10822.907439231873
2 .test_ctl_pd.shape[0]= 90
test.shape[0] = 19 train.shape[0]= 109
e_e0= 30 e_e1= 79
proportion= 1.6333333333333333
proportion= 0.8
time.time()-start_time= 0.9924025535583496
****** w= 4 j= 11 n= 2 prop= 0.8 Eps= 0.2 *********** acc 0.9473684210526315 acc_m 0.912280701754386 lb[0,0]= GLM_1_AutoML_20210621_223819 Execution time:  10901.36674451828
3 .test_ctl_pd.shape[0]= 71
test.shape[0] = 19 train.shape[0]= 109
e_e0= 30 

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


time.time()-start_time= 1.2187552452087402


H2OJobCancelled: ignored

In [ ]:
"while True:pass

KeyboardInterrupt: ignored